In [164]:
%matplotlib inline

In [165]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import matplotlib.style as style 

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import numpy as np
import re
import string

In [166]:
aggReports=pd.read_csv("../data/processed/clean_data.csv",index_col=0)
# aggReports.head()

C:\Users\Rajasvi\Anaconda3\envs\ece143\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.



In [167]:
expl_aggReports = pd.read_csv("../data/processed/exploded_out.csv",index_col=0)b

In [187]:
aggReports_prod = pd.read_csv("../data/processed/clean_brand_data.csv",index_col=0)
aggReports_prod['brand']

0          VALUED NATIONAL
1         DAVIDS SUNFLOWER
2                    KASHI
3                   DANNON
4                   COPPER
                ...       
131256            FISH OIL
131257      CENTRUM SILVER
131258      CENTRUM SILVER
131259      CENTRUM FLAVOR
131260      CENTRUM SILVER
Name: brand, Length: 131258, dtype: object

In [168]:
relv_outcomes = ["Death","Life Threatening","Hospitalization","Disability","Patient Visited ER"]

In [169]:
def brand_preprocess(x, first_word=True,trim_len=2):
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    x = regex.sub('', x)
    stop = stopwords.words('english')
    n_x = [_.upper() for _ in x.lower().split(" ") if _ not in stop]

    if len(n_x)==0:
        return ""
    if first_word:
        return n_x[0]
    return " ".join(n_x) if len(n_x)<trim_len else " ".join(n_x[:trim_len])


In [170]:

def brand_outcomes_plot( base_df,category,filename,title, relv_outcomes = ["Death","Life Threatening","Hospitalization","Disability","Patient Visited ER"],first_word=True,trim_len=2,):

    df = base_df[(base_df['category']==category) & (base_df['product']!="EXEMPTION 4")].copy()
    df.dropna(inplace=True)

    df['brand'] =df['product'].apply(brand_preprocess,first_word=first_word,trim_len=trim_len)
    g=df.groupby(['brand'])['report_id'].count().sort_values(ascending=False)
    

    relv_brands = list(g.reset_index()['brand'].values[:10])
    relv_df = df[df['outcomes'].isin(relv_outcomes)]
    relv_df = relv_df[relv_df['brand'].isin(relv_brands)]

    fig=px.histogram(relv_df,x="brand",color="outcomes",barmode="stack",histnorm="probability",title=f"{title} | Brands/Products vs Outcomes")
    # py.plot(fig)
    # fig=px.bar(df,x="brand",y="c")
    fig.show()

    top_brands_df = g.reset_index()[:10].rename(columns={'report_id':'#events'})
    fig_pie = px.pie(top_brands_df, values='#events', names='brand', title=f'{title} | Brand-wise Reported Adverse Events Count')
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.show()

    # plot(fig, filename =f"../reports/figures/brands_vs_outcomes_{filename}.html")

In [171]:
base_df=expl_aggReports
category="Cosmetics"
first_word=True
trim_len=2

df = base_df[(base_df['category']==category) & (base_df['product']!="EXEMPTION 4")].copy()
df.dropna(inplace=True)

df['brand'] =df['product'].apply(brand_preprocess,first_word=first_word,trim_len=trim_len)

# Which Cosmetics brand can *really* kill you? (not just your purse :P)

It is quite interesting to note that **Wen-L'Oreal-Brazilian Blowout** equally hold the responsilbity for causing the majority deaths cause due to cosmetics. Therefore among top brands, there's 1 out of 3 chances that you can die from *Wen/L'Oreal/Brazilian out*

In [172]:
# Cosmetics Brand Clustering
brand_outcomes_plot(expl_aggReports,"Cosmetics",filename="cosm",title="Cosmetics",relv_outcomes=relv_outcomes,first_word=True,trim_len=2)



In [173]:
# Vitamins Brand Clustering
brand_outcomes_plot(expl_aggReports,"Vit/Min/Prot/Unconv Diet(Human/Animal)",filename="vit",title="Vitamins/Minerals",relv_outcomes=relv_outcomes,first_word=False,trim_len=2)

In [174]:
# Vegetable Brand Clustering
brand_outcomes_plot(expl_aggReports,"Vegetables/Vegetable Products",relv_outcomes=relv_outcomes,filename="veg",title="Vegetable(s)",first_word=True)

In [175]:
# Nuts/Edible Brand Clustering
brand_outcomes_plot(expl_aggReports,"Nuts/Edible Seed",filename="nuts",title="Nuts",relv_outcomes=relv_outcomes,first_word=True)

In [176]:
# Plot category wise outcomes
df=expl_aggReports.dropna()

relv_categories = list(df.groupby("category")['report_id'].count().sort_values(ascending=False).reset_index()['category'].values[:5])

df = df[df['outcomes'].isin(relv_outcomes)]
df = df[df['category'].isin(relv_categories)]

In [177]:
fig=px.histogram(df,x="category",color="outcomes",barmode="stack",histnorm="probability")
fig.show()
# expl_aggReports.head()

In [178]:
df = base_df[(base_df['category']==category) & (base_df['product']!="EXEMPTION 4")].copy()
df.dropna(inplace=True)
df.head()

,report_id,product,category,outcomes
407,173153,BRAZILIAN BLOW ACAI SOLUTION,Cosmetics,Disability
408,173153,BRAZILIAN BLOW ACAI SOLUTION,Cosmetics,Medically Important
505,173247,WEN BY CHAZ DEAN,Cosmetics,Medically Important
556,173283,MAKE-UP REMOVER,Cosmetics,Other Outcome
557,173283,MASCARA,Cosmetics,Other Outcome


In [179]:
fig =go.Figure(go.Sunburst(
    labels=["Eve", "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents=["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
    values=[10, 14, 12, 10, 2, 6, 6, 4, 4],
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

In [180]:
import chart_studio.plotly as py
import chart_studio.presentation_objs as pres
import chart_studio

In [181]:
chart_studio.tools.set_credentials_file(username="rajasvi",api_key="dqJ4gJ68occfMuMAgSWH")
chart_studio.tools.set_config_file(world_readable=True,sharing='public')
